In [ ]:
import re

def to_audio_link_format_single_verse(single_verse):
    '''
    >>> to_audio_link_format_single_verse('1 John 5:1\u201312')
    '1John.5'
    '''
    
    book_transformations_dict = {
        'Genesis': 'Gen',
        'Exodus': 'Exod',
        'Leviticus': 'Lev',
        'Numbers': 'Num',
        'Deuteronomy': 'Deut',
        'Joshua': 'Josh',
        'Judges': 'Judg',
        'Ruth': 'Ruth',
        '1 Samuel': '1Sam',
        '2 Samuel': '2Sam',
        '1 Kings': '1Kgs',
        '2 Kings': '2Kgs',
        '1 Chronicles': '1Chr',
        '2 Chronicles': '2Chr',
        'Ezra': 'Ezra',
        'Nehemiah': 'Neh',
        'Esther': 'Esth',
        'Job': 'Job',
        'Psalm': 'Ps',
        'Proverbs': 'Prov',
        'Ecclesiastes': 'Eccl',
        'Song of Solomon': 'Song',
        'Isaiah': 'Isa',
        'Jeremiah': 'Jer',
        'Lamentations': 'Lam',
        'Ezekiel': 'Ezek',
        'Daniel': 'Dan',
        'Hosea': 'Hos',
        'Joel': 'Joel',
        'Amos': 'Amos',
        'Obadiah': 'Obad',
        'Jonah': 'Jonah',
        'Micah': 'Mic',
        'Nahum': 'Nah',
        'Habakkuk': 'Hab',
        'Zephaniah': 'Zeph',
        'Haggai': 'Hag',
        'Zechariah': 'Zech',
        'Malachi': 'Mal',
        'Matthew': 'Matt',
        'Mark': 'Mark',
        'Luke': 'Luke',
        'John': 'John',
        'Acts': 'Acts',
        'Romans': 'Rom',
        '1 Corinthians': '1Cor',
        '2 Corinthians': '2Cor',
        'Galatians': 'Gal',
        'Ephesians': 'Eph',
        'Philippians': 'Phil',
        'Colossians': 'Col',
        '1 Thessalonians': '1Thess',
        '2 Thessalonians': '2Thess',
        '1 Timothy': '1Tim',
        '2 Timothy': '2Tim',
        'Titus': 'Titus',
        'Philemon': 'Phlm',
        'Hebrews': 'Heb',
        'James': 'Jas',
        '1 Peter': '1Pet',
        '2 Peter': '2Pet',
        '1 John': '1John',
        '2 John': '2John',
        '3 John': '3John',
        'Jude': 'Jude',
        'Revelation': 'Rev'
    }

    result = single_verse
    
    # chop off the verse
    result = re.sub(':.*$', '', result)
    
    # find the best key
    matching_keys = [key for key in book_transformations_dict.keys() if key in result]
    matching_keys.sort(key=len)
    if len(matching_keys) > 0:
        longest_key = matching_keys[-1]
    else:
        return result
       
    #perform substitution
    result = re.sub(longest_key, book_transformations_dict[longest_key], result)
    
    # replace space with .
    result = result.replace(' ', '.')
    
    return result
    

def to_audio_link_format(verses_together):
    '''
    >>> to_audio_link_format('Psalm 103; Genesis 17:1\u201313; Romans 2:17\u201329')
    'Ps.103,Gen.17,Rom.2'
    '''
    split_verses = [verse.strip() for verse in verses_together.split(';')]
    processed_verses = [to_audio_link_format_single_verse(verse) for verse in split_verses] 
    return ','.join(processed_verses)
    

def to_reading_link_format(verses_together):
    '''
    >>> to_reading_link_format('Psalm 103; Genesis 17:1\u201313; Romans 2:17\u2013end')
    'Psalm+103%3B+Genesis+17:1-13%3B+Romans+2:17-'
    
    >>> to_reading_link_format('Titus 2:11-12,(13-14)')
    'Titus+2:11-12,%2813-14%29'
    '''
    result = verses_together.replace(' ', '+')
    result = result.replace('end', '')
    result = result.replace(';', '%3B')
    result = result.replace('(', '%28')
    result = result.replace(')', '%29')
    result = result.replace('\u2013','-') # remove em dash again
    return result

def clean(input):
    input = input.replace('*', '')
    input = input.replace('-', '\u2013') # ensure using em dash
    return input

def map_set_of_readings(raw_readings_string):
    
    if not raw_readings_string:
        return None
    
    cleaned_readings = clean(raw_readings_string)
    split_readings = [reading.strip() for reading in cleaned_readings.split(';')]
    
    readings = []
    for reading in split_readings:
        reading = {
            'pretty': reading,
            'readQS': to_reading_link_format(reading),
            'audioQS': to_audio_link_format(reading)
        }
        readings.append(reading)
            
    return {
        'readings' : readings,
        'readQS': to_reading_link_format(cleaned_readings),
        'audioQS': to_audio_link_format(cleaned_readings)
    }


In [ ]:
import csv
import json

json_dictionary_v2 = {}

with open('./input/all_readings.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
         
        day_dictionary = {}
        day_dictionary['datePretty'] = row['date_pretty']
        day_dictionary['calendar'] = row['day_description']
        day_dictionary['rclTrack1'] = map_set_of_readings(row['rcl_track_1'])
        day_dictionary['morning'] = map_set_of_readings(row['morning_prayers_together']) 
        day_dictionary['evening'] = map_set_of_readings(row['evening_prayers_together'])

        json_dictionary_v2[row['date']] = day_dictionary
        
print(json.dumps(json_dictionary_v2, indent=4))

In [ ]:
import doctest
doctest.testmod(verbose=True)

In [ ]:
with open('./output/data.ts', 'w') as outputfile:
    outputfile.write('import { IReadingsForDay } from "./interfaces";\r\r')
    outputfile.write('const Data: { [dateKey: string]: IReadingsForDay } = ')
    outputfile.write(json.dumps(json_dictionary_v2, indent=4) + ';\r\r')
    outputfile.write('export default Data;')

In [ ]:
with open('./output/last-generated-date.txt', 'w') as datefile:
    date = list(json_dictionary_v2)[-1].replace('/', '-')
    datefile.write(date)